###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [1]:
import pandas as pd

###Archivo 3cjd-phaj.csv

Contiene información sobre incautaciones de basuco.  Incluye el número de kilos de basuco incautados por la Fuerza Pública en el ejercicio de sus funciones. Entiéndase por Basuco como la droga obtenida a partir de la pasta o base de cocaína que se mezcla con diferentes agentes de corte con el fin de comercializarla para su consumo.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/3cjd-phaj.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,cantidad,unidad
0,2024-07-31T00:00:00.000,5,ANTIOQUIA,05266,ENVIGADO,0.012,KILOGRAMO
1,2024-07-31T00:00:00.000,5,ANTIOQUIA,05001,MEDELLIN,0.070,KILOGRAMO
2,2024-07-31T00:00:00.000,11,BOGOTÁ D.C.,11001,"BOGOTA, D.C.",0.120,KILOGRAMO
3,2024-07-31T00:00:00.000,11,BOGOTÁ D.C.,11001,"BOGOTA, D.C.",0.030,KILOGRAMO
4,2024-07-31T00:00:00.000,11,BOGOTÁ D.C.,11001,"BOGOTA, D.C.",0.150,KILOGRAMO


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439542 entries, 0 to 439541
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   fecha_hecho   439542 non-null  object 
 1   cod_depto     439542 non-null  int64  
 2   departamento  439542 non-null  object 
 3   cod_muni      439542 non-null  object 
 4   municipio     439542 non-null  object 
 5   cantidad      439542 non-null  float64
 6   unidad        439542 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 23.5+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad', 'unidad']
basuco_confiscations_MinDefensa = df[relevant_cols]
basuco_confiscations_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439542 entries, 0 to 439541
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   cod_muni      439542 non-null  object 
 1   departamento  439542 non-null  object 
 2   municipio     439542 non-null  object 
 3   fecha_hecho   439542 non-null  object 
 4   cantidad      439542 non-null  float64
 5   unidad        439542 non-null  object 
dtypes: float64(1), object(5)
memory usage: 20.1+ MB


### Convertir columna fecha_hecho a tipo date

In [5]:
# Hacer una copia explícita del DataFrame
basuco_confiscations_MinDefensa_ = basuco_confiscations_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
basuco_confiscations_MinDefensa_['fecha_hecho'] = pd.to_datetime(basuco_confiscations_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
basuco_confiscations_MinDefensa_['fecha_hecho'] = basuco_confiscations_MinDefensa_['fecha_hecho'].dt.year

In [6]:
basuco_confiscations_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010], dtype=int32)

### Verificar valores nulos

In [7]:
basuco_confiscations_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,0
municipio,0
fecha_hecho,0
cantidad,0
unidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [9]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'unidad']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(basuco_confiscations_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['ANTIOQUIA' 'BOGOTÁ D.C.' 'CALDAS' 'HUILA' 'LA GUAJIRA' 'MAGDALENA'
 'QUINDÍO' 'RISARALDA' 'SANTANDER' 'VALLE DEL CAUCA' 'BOLÍVAR'
 'CUNDINAMARCA' 'META' 'NARIÑO' 'TOLIMA' 'ARAUCA' 'BOYACÁ' 'GUAVIARE'
 'NORTE DE SANTANDER' 'CASANARE' 'CESAR' 'ATLÁNTICO' 'CAUCA' 'CÓRDOBA'
 'SUCRE' 'PUTUMAYO' 'CAQUETÁ' 'GUAINÍA' 'SAN ANDRÉS ISLAS' 'CHOCÓ'
 'AMAZONAS' 'BOGOTA D.C.' 'BOLIVAR' 'ATLANTICO' 'QUINDIO' 'BOYACA'
 'CORDOBA' 'CAQUETA' 'SAN ANDRES ISLAS' 'CHOCO' 'VICHADA' 'VAUPES'
 'GUAINIA' 'VAUPÉS']

Categorías en la columna 'municipio':
['ENVIGADO' 'MEDELLIN' 'BOGOTA, D.C.' 'MANIZALES' 'RIOSUCIO' 'ACEVEDO'
 'FONSECA' 'RIOHACHA' 'SANTA MARTA' 'LA TEBAIDA' 'DOSQUEBRADAS' 'BOLIVAR'
 'PIEDECUESTA' 'TRUJILLO' 'MAGANGUE' 'SAN ESTANISLAO' 'LA DORADA'
 'GIRARDOT' 'MOSQUERA' 'VILLAVICENCIO' 'IPIALES' 'BALBOA' 'PEREIRA'
 'BUCARAMANGA' 'LERIDA' 'MARIQUITA' 'CARTAGO' 'PALMIRA' 'RESTREPO'
 'BARBOSA' 'BELLO' 'CALDAS' 'LA ESTRELLA' 'SANTAFE DE ANTIOQUIA' 'TAME'
 'EL CA

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [10]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [11]:
# Aplicar la función a todas las columnas categóricas
basuco_confiscations_MinDefensa_[categorical_col] = basuco_confiscations_MinDefensa_[categorical_col].astype(str)
for col in categorical_col:
    basuco_confiscations_MinDefensa_[col] = basuco_confiscations_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [12]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df basuco_confiscations_MinDefensa_

In [14]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
basuco_confiscations_MinDefensa_['cod_muni'] = basuco_confiscations_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = basuco_confiscations_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [15]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [16]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,439542,1.0


In [17]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = basuco_confiscations_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    05266
1    05001
2    11001
3    11001
4    11001



Nota: El 100%  de los codigos que identifican a los municipios en el df basuco_confiscations_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [18]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [19]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [20]:
# Comparar listas de departamento
compare_lists(basuco_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en basuco_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en basuco_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'SAN ANDRES ISLAS'}


  - Solucionar discrepancias en nombres de departamento

In [21]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES ISLAS': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe basuco_confiscations_MinDefensa_
basuco_confiscations_MinDefensa_['departamento'] = basuco_confiscations_MinDefensa_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [22]:
# Comparar listas de departamento
compare_lists(basuco_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en basuco_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en basuco_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [23]:
# Comparar listas de códigos
compare_lists(basuco_confiscations_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en basuco_confiscations_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en basuco_confiscations_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [25]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_basuco_confiscations_MinDefensa = basuco_confiscations_MinDefensa_.drop(columns=columns_to_drop)

In [27]:
# Adicionar columna para trazabilidad de la fuente
final_basuco_confiscations_MinDefensa['source_id'] = 31

In [28]:
final_basuco_confiscations_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'cantidad', 'unidad', 'source_id'], dtype='object')

In [29]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'unidad': 'unit',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_basuco_confiscations_MinDefensa.rename(columns=translation_map, inplace=True)

In [30]:
#Estructura final del dataset a integrar a la base de datos
final_basuco_confiscations_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439542 entries, 0 to 439541
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   dane_code         439542 non-null  object 
 1   year_of_incident  439542 non-null  int32  
 2   amount            439542 non-null  float64
 3   unit              439542 non-null  object 
 4   source_id         439542 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 15.1+ MB


## Salvar en archivo csv en el drive

In [31]:
# Guardar en archivos CSV en el drive
final_basuco_confiscations_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/basuco_confiscations_MinDefensa.csv', index=False)